In [1]:
import json
import ast
import pandas as pd
import os
import regex

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import os
import sys
import numpy as np

parent_directory = os.getcwd()

### Read comments from json files

In [4]:
import glob
# Define the directory path
directory = f"{parent_directory}/data/reviews"

# Initialize an empty list to store the data
data = []

# Get a list of all JSON files in the directory and its sub-folders
json_files = glob.glob(directory + "/**/*.json", recursive=True)

# Iterate over all JSON files
for file_path in json_files:
    # Read the JSON file and append its contents to the data list
    with open(file_path, "r") as file:
        file_content = file.read()
        try:
            # Try to parse the content as JSON
            json_data = json.loads(file_content)
            data.append(json_data)
        except json.JSONDecodeError:
            try:
                # If parsing as JSON fails, try to evaluate the content as a Python dictionary
                dict_data = ast.literal_eval(file_content)
                # Convert the dictionary to a JSON string
                json_data = json.dumps(dict_data)
                data.append(json.loads(json_data))
            except (ValueError, SyntaxError) as e:
                print(f"Error parsing JSON from file {file_path}: {e}")

# Create a dataframe from the data list
df = pd.DataFrame(data)

### Remove Unnecessary Columns

In [5]:
df.columns

Index(['id', 'title', 'content', 'status', 'thank_count', 'score', 'new_score',
       'customer_id', 'comment_count', 'rating', 'images', 'thanked',
       'created_at', 'created_by', 'suggestions', 'attributes',
       'product_attributes', 'spid', 'is_photo', 'seller', 'product_id',
       'timeline', 'vote_attributes', 'delivery_rating', 'content_id'],
      dtype='object')

In [6]:
kept_cols_lv1 = ['id', 'title', 'content', 'status', 'rating', 
             'suggestions', 'vote_attributes', 'delivery_rating']

df = df[kept_cols_lv1]
df = df[df.status == 'approved']
df = df[df.content != ""]
df = df[df['content'].str.len() > 1]

kept_cols_lv2 = ['id', 'title', 'content', 'rating']
df = df[kept_cols_lv2]

In [7]:
df

,id,title,content,rating
0,15887644,Cực kì hài lòng,tốt,5
1,19569865,Cực kì hài lòng,Good,5
3,15803979,Cực kì hài lòng,"Ổn nhưng hơi cứng, con gái khó uốn, đt nhẹ cở ...",4
5,17357926,Cực kì hài lòng,Hài lòng,5
7,15009837,Hài lòng,"Nói chung là ổn, tuy nhiên màu chưa được giống...",4
...,...,...,...,...
24262,9160824,Cực kì hài lòng,Không có gì để chê. Giá lúc mua cũng rẻ,5
24263,16300584,Rất không hài lòng,giao hàng cường lực vỡ nát.h ko xài đc,1
24264,17735535,Cực kì hài lòng,"Nói chung là tạm được, máy chưa có tiếng Việt....",5
24265,16746567,Cực kì hài lòng,"Giao hàng hơi lâu. sản phẩm còn nguyên seal, đ...",5


#### Preprocess data

In [8]:
from utils.preprocess import (
    remove_HTML,
    convert_unicode,
    standardize_sentence_typing,
    normalize_acronyms, 
    word_segmentation, # When use PhoBERT
    remove_unnecessary_characters,
    remove_redundant_newlines
) 

2024-05-12 14:12:25 INFO  WordSegmenter:24 - Loading Word Segmentation model


In [9]:
def text_preprocess(text):
    text = remove_HTML(text)
    text = convert_unicode(text) 
    text = standardize_sentence_typing(text)
    text = normalize_acronyms(text)
    text = word_segmentation(text) # When use PhoBERT
    text = remove_unnecessary_characters(text)
    text = remove_redundant_newlines(text)
    # return text.lower()
    return text

In [10]:
df['processed_content'] = df['content'].apply(text_preprocess)

In [11]:
output_cols = ['id', 'title', 'content', 'processed_content', 'rating']
df = df[output_cols]
df

,id,title,content,processed_content,rating
0,15887644,Cực kì hài lòng,tốt,tốt,5
1,19569865,Cực kì hài lòng,Good,tốt,5
3,15803979,Cực kì hài lòng,"Ổn nhưng hơi cứng, con gái khó uốn, đt nhẹ cở ...",ổn nhưng hơi cứng con gái khó uốn điện_thoại n...,4
5,17357926,Cực kì hài lòng,Hài lòng,hài_lòng,5
7,15009837,Hài lòng,"Nói chung là ổn, tuy nhiên màu chưa được giống...",nói_chung là ổn tuy_nhiên màu chưa được giống ...,4
...,...,...,...,...,...
24262,9160824,Cực kì hài lòng,Không có gì để chê. Giá lúc mua cũng rẻ,không có gì để chê giá lúc mua cũng rẻ,5
24263,16300584,Rất không hài lòng,giao hàng cường lực vỡ nát.h ko xài đc,giao hàng cường_lực vỡ nát h không xài được,1
24264,17735535,Cực kì hài lòng,"Nói chung là tạm được, máy chưa có tiếng Việt....",nói_chung là tạm được máy chưa có tiếng việt n...,5
24265,16746567,Cực kì hài lòng,"Giao hàng hơi lâu. sản phẩm còn nguyên seal, đ...",giao hàng hơi lâu sản_phẩm còn nguyên seal đầy...,5


Remove duplicated contents

In [12]:
df[df['processed_content'] == 'tốt']

,id,title,content,processed_content,rating
0,15887644,Cực kì hài lòng,tốt,tốt,5
1,19569865,Cực kì hài lòng,Good,tốt,5
34,19714417,Cực kì hài lòng,Tốt,tốt,5
105,17103865,Cực kì hài lòng,tốt,tốt,5
472,14270950,Cực kì hài lòng,Good,tốt,5
...,...,...,...,...,...
23867,3579617,Hài lòng,Good,tốt,4
24038,5888518,Hài lòng,tot,tốt,4
24063,19502107,Cực kì hài lòng,Tốt,tốt,5
24097,6522954,Cực kì hài lòng,Tốt?,tốt,5


In [13]:
df.drop_duplicates(subset='processed_content', inplace=True)

In [14]:
df[df['processed_content'] == 'tốt']

,id,title,content,processed_content,rating
0,15887644,Cực kì hài lòng,tốt,tốt,5


### Review label distribution

In [15]:
df['rating'].value_counts()

rating
5    8202
1    2360
4    1744
3     998
2     744
Name: count, dtype: int64

In [16]:
df.loc[:, 'sentiment_3_cls'] = df['rating'].apply(lambda x: 'neutral' if x == 3 else 'negative' if x < 3 else 'positive')
df.loc[:, 'sentiment_2_cls'] = df['rating'].apply(lambda x: 'negative' if x < 3 else 'positive')

In [17]:
df

,id,title,content,processed_content,rating,sentiment_3_cls,sentiment_2_cls
0,15887644,Cực kì hài lòng,tốt,tốt,5,positive,positive
3,15803979,Cực kì hài lòng,"Ổn nhưng hơi cứng, con gái khó uốn, đt nhẹ cở ...",ổn nhưng hơi cứng con gái khó uốn điện_thoại n...,4,positive,positive
5,17357926,Cực kì hài lòng,Hài lòng,hài_lòng,5,positive,positive
7,15009837,Hài lòng,"Nói chung là ổn, tuy nhiên màu chưa được giống...",nói_chung là ổn tuy_nhiên màu chưa được giống ...,4,positive,positive
8,19110097,Bình thường,điện thoại tốt\r\nnhưng tai nghe khuyến mãi kè...,điện_thoại tốt nhưng tai nghe khuyến_mãi kèm t...,3,neutral,positive
...,...,...,...,...,...,...,...
24242,12765989,Rất không hài lòng,Thiết kế vầy sao mà xài được trời,thiết_kế vầy sao mà xài được trời,1,negative,negative
24248,16557176,Rất không hài lòng,quá thất vọng cho sản phẩm lần này.,quá thất_vọng cho sản_phẩm lần này,1,negative,negative
24253,17480824,Bình thường,"San phẩm xài chất lượng như mong được, kích th...",san phẩm xài chất_lượng như mong được kích_thư...,3,neutral,positive
24262,9160824,Cực kì hài lòng,Không có gì để chê. Giá lúc mua cũng rẻ,không có gì để chê giá lúc mua cũng rẻ,5,positive,positive


In [18]:
df['sentiment_3_cls'].value_counts()

sentiment_3_cls
positive    9946
negative    3104
neutral      998
Name: count, dtype: int64

In [19]:
df['sentiment_3_cls'].value_counts(normalize=True) * 100

sentiment_3_cls
positive    70.800114
negative    22.095672
neutral      7.104214
Name: proportion, dtype: float64

### Split train, valid, test

In [20]:
from sklearn.model_selection import train_test_split

# Split the dataset into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Split the train set into train and validation sets
train_df, valid_df = train_test_split(train_df, test_size=0.2, random_state=42)

# Print the shapes of the resulting datasets
print("Train set shape:", train_df.shape)
print("Validation set shape:", valid_df.shape)
print("Test set shape:", test_df.shape)

Train set shape: (8990, 7)
Validation set shape: (2248, 7)
Test set shape: (2810, 7)


In [21]:
# Calculate the sentiment_3_cls distribution for each set
train_distribution = train_df['sentiment_3_cls'].value_counts()
valid_distribution = valid_df['sentiment_3_cls'].value_counts()
test_distribution = test_df['sentiment_3_cls'].value_counts()

# Calculate the percentage distribution for each set
train_percentage = train_df['sentiment_3_cls'].value_counts(normalize=True) * 100
valid_percentage = valid_df['sentiment_3_cls'].value_counts(normalize=True) * 100
test_percentage = test_df['sentiment_3_cls'].value_counts(normalize=True) * 100

# Create a single dataframe to compare the distributions
comparison_df = pd.concat([train_distribution, train_percentage, valid_distribution, valid_percentage, test_distribution, test_percentage], axis=1)
comparison_df.columns = ['Train Set (Count)', 'Train Set (%)', 'Validation Set (Count)', 'Validation Set (%)', 'Test Set (Count)', 'Test Set (%)']
comparison_df = comparison_df.fillna(0)

# Display the comparison dataframe
comparison_df


,Train Set (Count),Train Set (%),Validation Set (Count),Validation Set (%),Test Set (Count),Test Set (%)
sentiment_3_cls,,,,,,
positive,6334,70.456062,1570,69.839858,2042,72.669039
negative,1996,22.202447,523,23.265125,585,20.818505
neutral,660,7.341491,155,6.895018,183,6.512456


In [22]:
train_df.to_csv(f'{parent_directory}/processed_data/review_electronic_devices_train.csv', index=False)
valid_df.to_csv(f'{parent_directory}/processed_data/review_electronic_devices_valid.csv', index=False)
test_df.to_csv(f'{parent_directory}/processed_data/review_electronic_devices_test.csv', index=False)

### Save as model input format

In [23]:
def format_and_save_output(df, path):
    sentences = []
    labels = []
    lengths = []
    for idx, row in df.iterrows():
        content = str(row['processed_content'])
        lengths.append(len(content.split()))
        label = row['sentiment_3_cls']
        sentences.append(content)
        labels.append(label)

    with open(f"{path}/seq.in", 'w') as f:
        f.write("\n".join(sentences))
    
    with open(f"{path}/label", 'w') as f:
        f.write("\n".join(labels))

    print(f"- {path.split('/')[-1]} set: ")
    print("  + average length: ", int(sum(lengths) / len(lengths)))
    print("  + max length: ", max(lengths))
    print("  + p95 length: ", int(np.percentile(lengths, 95)))
    print("  + p99 length: ", int(np.percentile(lengths, 99)))

In [24]:
format_and_save_output(train_df, f"{parent_directory}/processed_data/word-level/train")
format_and_save_output(valid_df, f"{parent_directory}/processed_data/word-level/dev")
format_and_save_output(test_df, f"{parent_directory}/processed_data/word-level/test")

- train set: 
  + average length:  22
  + max length:  638
  + p95 length:  64
  + p99 length:  116
- dev set: 
  + average length:  22
  + max length:  358
  + p95 length:  62
  + p99 length:  114
- test set: 
  + average length:  22
  + max length:  554
  + p95 length:  64
  + p99 length:  110


In [25]:
df.shape

(14048, 7)